In [1]:
import pandas as pd
import numpy as np
import glob

In [44]:
all_files=glob.glob(r'C:\Users\飘哥\Desktop\GPS数据\20190316\*.dt')
all_data_frame = []
len(all_files)

1440

In [45]:
for i in all_files:
    data=pd.read_table(i,names=['DATE','Stime','VehicleNum','Lng','Lat','OpenStatus','direct','Speed','SEVERID'],header=None,sep='\s')
    all_data_frame.append(data)
data_frame_concat = pd.concat(all_data_frame,ignore_index=True, sort=True)#将列表都合并并转化为dataframe    

C:\Anaconda\lib\site-packages\pandas\io\parsers.py:767: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  return read_csv(**locals())


In [46]:
data_frame_concat

,DATE,Lat,Lng,OpenStatus,SEVERID,Speed,Stime,VehicleNum,direct
0,2019-03-15,31.306112,121.476685,0,4,35.0,23:59:58,26182,4
1,2019-03-15,30.715600,121.346400,0,2,0.0,23:59:58,136782,7
2,2019-03-15,31.321600,121.272200,0,2,0.0,23:59:59,148875,0
3,2019-03-15,31.217000,121.483000,0,2,0.0,23:59:59,143547,0
4,2019-03-15,31.171400,121.531100,1,2,44.0,23:59:58,113729,4
...,...,...,...,...,...,...,...,...,...
45198248,2019-03-16,31.199000,121.440100,0,2,0.0,23:59:58,144864,0
45198249,2019-03-16,31.517233,121.730280,0,5,0.0,23:59:55,171839,5
45198250,2019-03-16,31.138475,121.657135,0,5,36.0,23:59:55,161308,2
45198251,2099-11-30,0.000000,0.000000,1,5,0.0,00:00:00,160111,0


In [47]:
data=data_frame_concat.query('DATE=="2019-03-16" and OpenStatus!=2')
data=data.sort_values(['VehicleNum','Stime'])
#这里，把上述异常数据清洗出来
#用到的条件是：
#1.后一位和前一位相等
#2.但是后一位与中间一位不等
#3.前一条数据，后一条数据的车牌相等
#4.中间一条数据，后一条数据的车牌相等

#各条件之间的交集可以用
#data[(条件1)&(条件2)]
#各条件之间的并集可以用
#data[(条件1)|(条件2)]
data = data[-((data['OpenStatus'].shift(-1) == data['OpenStatus'].shift())&
(data['OpenStatus'].shift(-1) != data['OpenStatus'])&
(data['VehicleNum'].shift(-1) == data['VehicleNum'].shift())&
(data['VehicleNum'].shift(-1) == data['VehicleNum']))]
data

,DATE,Lat,Lng,OpenStatus,SEVERID,Speed,Stime,VehicleNum,direct
82563,2019-03-16,31.120900,121.45430,1,2,0.0,00:02:38,141,6
112699,2019-03-16,31.120900,121.45430,1,2,0.0,00:03:38,141,5
143006,2019-03-16,31.120900,121.45430,1,2,0.0,00:04:38,141,4
173164,2019-03-16,31.120900,121.45430,1,2,0.0,00:05:38,141,6
203388,2019-03-16,31.120900,121.45430,1,2,0.0,00:06:38,141,6
...,...,...,...,...,...,...,...,...,...
39829601,2019-03-16,31.108896,121.34826,0,5,0.0,21:19:06,953530,0
39863828,2019-03-16,31.108896,121.34826,0,5,0.0,21:20:19,953530,0
39886071,2019-03-16,31.108898,121.34826,0,5,0.0,21:20:59,953530,0
39920127,2019-03-16,31.108898,121.34826,0,5,0.0,21:21:59,953530,0


# 识别OD

In [48]:
data['OpenStatus1']=data['OpenStatus'].shift(-1)
data['StatusChange']=data['OpenStatus1']-data['OpenStatus']
data

,DATE,Lat,Lng,OpenStatus,SEVERID,Speed,Stime,VehicleNum,direct,OpenStatus1,StatusChange
82563,2019-03-16,31.120900,121.45430,1,2,0.0,00:02:38,141,6,1.0,0.0
112699,2019-03-16,31.120900,121.45430,1,2,0.0,00:03:38,141,5,1.0,0.0
143006,2019-03-16,31.120900,121.45430,1,2,0.0,00:04:38,141,4,1.0,0.0
173164,2019-03-16,31.120900,121.45430,1,2,0.0,00:05:38,141,6,1.0,0.0
203388,2019-03-16,31.120900,121.45430,1,2,0.0,00:06:38,141,6,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
39829601,2019-03-16,31.108896,121.34826,0,5,0.0,21:19:06,953530,0,0.0,0.0
39863828,2019-03-16,31.108896,121.34826,0,5,0.0,21:20:19,953530,0,0.0,0.0
39886071,2019-03-16,31.108898,121.34826,0,5,0.0,21:20:59,953530,0,0.0,0.0
39920127,2019-03-16,31.108898,121.34826,0,5,0.0,21:21:59,953530,0,0.0,0.0


In [49]:
#把statuschange为1和-1的筛选出来并且上下车车牌号要相同,-1下车,1上车
oddata = data[((data['StatusChange'] == 1)|(data['StatusChange'] == -1))
&(data['VehicleNum'] == data['VehicleNum'].shift(-1))]
oddata

,DATE,Lat,Lng,OpenStatus,SEVERID,Speed,Stime,VehicleNum,direct,OpenStatus1,StatusChange
8766842,2019-03-16,31.120900,121.454200,1,2,0.0,05:47:39,141,6,0.0,-1.0
17301842,2019-03-16,31.122500,121.457100,0,2,8.0,10:30:21,141,0,1.0,1.0
17929078,2019-03-16,31.184900,121.452900,1,2,4.0,10:48:21,141,5,0.0,-1.0
18208213,2019-03-16,31.175700,121.452800,0,2,10.0,10:56:21,141,0,1.0,1.0
18663207,2019-03-16,31.187700,121.439200,1,2,0.0,11:09:21,141,5,0.0,-1.0
...,...,...,...,...,...,...,...,...,...,...,...
12525202,2019-03-16,31.212786,121.426480,1,5,0.0,08:08:51,953530,0,0.0,-1.0
12563291,2019-03-16,31.212818,121.426400,0,5,3.0,08:10:01,953530,0,1.0,1.0
13025077,2019-03-16,31.180975,121.375470,1,5,32.0,08:24:18,953530,0,0.0,-1.0
13063182,2019-03-16,31.180426,121.375534,0,5,16.0,08:25:27,953530,0,1.0,1.0


In [50]:
oddata=oddata[['VehicleNum','Stime','Lng','Lat','StatusChange']]
oddata.columns=['VehicleNum','Stime','SLng','SLat','StatusChange']
oddata['ELng']=oddata['SLng'].shift(-1)
oddata['ELat']=oddata['SLat'].shift(-1)
oddata['Etime']=oddata['Stime'].shift(-1)
oddata

C:\Anaconda\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Anaconda\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
C:\Anaconda\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = val

,VehicleNum,Stime,SLng,SLat,StatusChange,ELng,ELat,Etime
8766842,141,05:47:39,121.454200,31.120900,-1.0,121.457100,31.122500,10:30:21
17301842,141,10:30:21,121.457100,31.122500,1.0,121.452900,31.184900,10:48:21
17929078,141,10:48:21,121.452900,31.184900,-1.0,121.452800,31.175700,10:56:21
18208213,141,10:56:21,121.452800,31.175700,1.0,121.439200,31.187700,11:09:21
18663207,141,11:09:21,121.439200,31.187700,-1.0,121.438600,31.186400,11:13:10
...,...,...,...,...,...,...,...,...
12525202,953530,08:08:51,121.426480,31.212786,-1.0,121.426400,31.212818,08:10:01
12563291,953530,08:10:01,121.426400,31.212818,1.0,121.375470,31.180975,08:24:18
13025077,953530,08:24:18,121.375470,31.180975,-1.0,121.375534,31.180426,08:25:27
13063182,953530,08:25:27,121.375534,31.180426,1.0,121.319695,31.189410,08:38:24


In [51]:
#最终OD处理数据
final=oddata[(oddata['StatusChange']==1)&(oddata['VehicleNum']==oddata['VehicleNum'].shift(-1))].drop('StatusChange',1)
final

,VehicleNum,Stime,SLng,SLat,ELng,ELat,Etime
17301842,141,10:30:21,121.457100,31.122500,121.452900,31.184900,10:48:21
18208213,141,10:56:21,121.452800,31.175700,121.439200,31.187700,11:09:21
18795957,141,11:13:10,121.438600,31.186400,121.423900,31.192800,11:24:30
19299872,141,11:27:42,121.424000,31.193000,121.412000,31.189700,11:34:42
19880762,141,11:44:36,121.423200,31.183600,121.444500,31.160700,11:54:36
...,...,...,...,...,...,...,...
9545474,953530,06:21:48,121.437870,31.222970,121.446530,31.247326,06:37:10
10468892,953530,06:57:55,121.416245,31.236212,121.438540,31.187090,07:13:35
12159049,953530,07:57:25,121.417915,31.229837,121.426480,31.212786,08:08:51
12563291,953530,08:10:01,121.426400,31.212818,121.375470,31.180975,08:24:18


In [52]:
final.to_csv(r'C:\Users\飘哥\Desktop\GPS数据\2019-03-16.csv',index=None,encoding='utf-8_sig')